<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/konstrukcja_mapy_pustyn/konstrukcja_mapy_pustyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import i instalacja potrzebnych bibliotek:

In [1]:
!pip install datashader
!pip install holoviews hvplot colorcet
!pip install geoviews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.2/511.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 45.2 MB/s eta 0:00:00


In [2]:
# import doinstalowanych pakietów
import datashader as ds
#import datashader.transfer_functions as tf
import colorcet as cc
import holoviews as hv
#from holoviews.operation.datashader import datashade
import geoviews as gv
import geoviews.tile_sources as gts
from holoviews import opts

In [3]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2024Project.git
%cd BigData2024Project
!git checkout refactoring-sprint2
%cd ..

Cloning into 'BigData2024Project'...
remote: Enumerating objects: 1430, done.
remote: Counting objects: 100% (776/776), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 1430 (delta 510), reused 527 (delta 344), pack-reused 654
Receiving objects: 100% (1430/1430), 55.24 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (773/773), done.
/content/BigData2024Project
Branch 'refactoring-sprint2' set up to track remote branch 'refactoring-sprint2' from 'origin'.
Switched to a new branch 'refactoring-sprint2'
/content


In [5]:
%cd BigData2024Project
!git pull
%cd ..

/content/BigData2024Project
Already up to date.
/content


In [6]:
%cd BigData2024Project
!git status
%cd ..

/content/BigData2024Project
On branch refactoring-sprint2
Your branch is up to date with 'origin/refactoring-sprint2'.

nothing to commit, working tree clean
/content


In [7]:
!chmod 755 /content/BigData2024Project/src/setup.sh
!/content/BigData2024Project/src/setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 41.8 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninsta

In [8]:
import sys
sys.path.append('/content/BigData2024Project/src')

In [9]:
from start_spark import initialize_spark
initialize_spark()

In [10]:
from pyspark.sql import SparkSession
from big_mess.loaders import preprocessed_loader

Wczytanie danych:

In [11]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
data=preprocessed_loader(spark,'/content/drive/MyDrive/BigMess/NASA/NASA_heuristic.csv')

In [13]:
data.show(5)

+---------+-------+------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+--------+
|      lon|    lat| Rainf|    Evap|AvgSurfT|  Albedo|SoilT_10_40cm|     GVEG| PotEvap|RootMoist|SoilM_100_200cm|SoilT_40_100cm|Year|Month|Pustynia|
+---------+-------+------+--------+--------+--------+-------------+---------+--------+---------+---------------+--------------+----+-----+--------+
|-112.0625|25.0625|1.1386|11.99051|288.2954|41.47715|     289.6995|0.1971279|137.6446| 398.8725|       208.2797|          NULL|2023|    1|       1|
|-111.9375|25.0625| 1.159|11.40101|287.4547|41.50941|     289.5214| 0.198604|152.8735| 375.6453|       192.4112|          NULL|2023|    1|       1|
|-111.8125|25.0625| 1.179|11.10241|287.6041|41.50537|      289.974|0.1711854|109.3242| 183.8275|       211.5441|          NULL|2023|    1|       1|
|-111.6875|25.0625|1.2057|11.20511|287.3168|41.50134|     289.8952|0.1711854|113.2493| 182.3848|       209.8121|

Konwersja na pandasowy dataframe, wyodrębniając przy tym jedynie 3 interesujące nas kolumny:

In [14]:
df = data.select("lon", "lat", "Pustynia").toPandas()

In [15]:
df

,lon,lat,Pustynia
0,-112.0625,25.0625,1
1,-111.9375,25.0625,1
2,-111.8125,25.0625,1
3,-111.6875,25.0625,1
4,-111.5625,25.0625,1
...,...,...,...
76058,-67.5625,52.9375,1
76059,-67.4375,52.9375,1
76060,-67.3125,52.9375,1
76061,-67.1875,52.9375,1


Funkcja służąca stworzeniu mapy w formacie .html:

In [16]:
def plot_map(df: pd.DataFrame, parameter_name: str, colormap: mpl.colors.LinearSegmentedColormap,
             point_size: int = 8, width: int = 900, height: int = 600, alpha: float = 1,
             bgcolor: str = 'white'):

    gdf = gv.Points(df, ['lon', 'lat'], [parameter_name]) # obiekt zawierający punkty
    tiles = gts.OSM # wybór mapy tła, w tym wypadku OpenStreetMap

    # łączenie mapy tła z punktami i ustawienie wybranych parametrów wizualizacji
    map_with_points = tiles * gdf.opts(
        color=parameter_name,
        cmap=colormap,
        size=point_size,
        width=width,
        height=height,
        colorbar=True,
        toolbar='above',
        tools=['hover', 'wheel_zoom', 'reset'],
        alpha=alpha # przezroczystość
    )

    map_with_points.opts(bgcolor=bgcolor)

    # zapis mapy do pliku .html
    output_filename = f'/content/drive/MyDrive/BigMess/NASA/output_map_{parameter_name}.html'
    hv.save(map_with_points, output_filename)

Stworzenie mapy dla uzyskanych danych:

In [17]:
plot_map(df=df, parameter_name='Pustynia', colormap=dict(zip(['1', '0'], ['yellow', 'green'])), point_size=3, alpha=0.7)

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
